# Agrupamento e Redução de Dimensionalidade
## utilizando o algoritmo k-means
### Aluno: Rayana Souza Rocha
[Link do do Github da atividade de Rayana](https://github.com/rayanarocha/machinelearning/blob/fdb4b3dcfb4a3878cc60be43717c383f5e0d1925/previs%C3%A3o-de-vota%C3%A7%C3%A3o-de-deputados/novo.ipynb)

#### O que é K-Means?

K-Means é um algoritmo de clusterização (ou agrupamento) disponível na biblioteca Scikit-Learn.

É um algoritmo de aprendizado não supervisionado (ou seja, que não precisa de inputs de confirmação externos) que avalia e clusteriza os dados de acordo com suas características, como por exemplo:

lojas/centro logistico
clientes/produtos ou serviços semelhantes
clientes/características semelhantes
séries/gênero da série ou faixa etaria
usuarios de uma rede social/usuario influenciador
paciente/sintoma ou característica semelhante

Fonte: https://medium.com/programadores-ajudando-programadores/k-means-o-que-%C3%A9-como-funciona-aplica%C3%A7%C3%B5es-e-exemplo-em-python-6021df6e2572

### Como funciona?

![k-means.png](k-means.png)

1. Primeiro, preciso definir um ‘K’, ou seja, um número de clusters (ou agrupamentos).
2. Depois, preciso definir, aleatoriamente, um centroide para cada cluster.
3. O próximo passo é calcular, para cada ponto, o centroide de menor distância. Cada ponto pertencerá ao centroide mais próximo (lembrar do exemplo do CD logístico e das lojas: cada loja (ponto) deve ser atendida pelo CD (centróide) mais próximo)
4. Agora, devo reposicionar o centróide. A nova posição do centroide deve ser a média da posição de todos os pontos do cluster.
5. Os dois ultimos passos são repetidos, iterativamente, até obtermos a posição ideal dos centróides.

In [41]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
#Istalando `stopwords` em outros idiomas
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_sertanejo = pd.read_csv('letras-ptbr-sertanejo-grande.csv')
print('\nFormato do dataset test:',df_sertanejo.shape, '\n')
df_sertanejo.sample(3)


Formato do dataset test: (15381, 7) 



,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
5386,Que Tal,"Você pensou que era bom, viu que era melhor. S...",Guilherme e Santiago,237,3.0,Sertanejo,Sertanejo; Romântico; Pop/Rock; Pop
3032,Exército de irmãos,Quantos de nós os nossos inimigos terão. Que e...,Daniel e Samuel,247,3.2,Sertanejo,Sertanejo; Gospel/Religioso; Romântico; Instru...
621,Jeito Natural,"Impulso diferente, pupila dilatada. Aposto que...",Breno e Caio Cesar,79,0.7,Sertanejo,Sertanejo; Country; Romântico; House; Funk Car...


In [14]:
df_sertanejo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15381 entries, 0 to 15380
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SName       15381 non-null  object 
 1   Lyric       15381 non-null  object 
 2   Artist      15381 non-null  object 
 3   Songs       15381 non-null  int64  
 4   Popularity  15381 non-null  float64
 5   Genre       15381 non-null  object 
 6   Genres      15381 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 841.3+ KB


#### Descrição dos dados:

- SName: Coluna que contém o título da música
- Lyric: Coluna que contém a letra da música
- Artist: Nome do artista ou banda
- Songs: Número de músicas do artista. Caso o mesmo artista tenha mais de uma música no conjunto, o valor dessa coluna estará repetido em todas as músicas.
- Popularity: Popularidade do artista. Caso o mesmo artista tenha mais de uma música no conjunto, o valor dessa coluna estará repetido em todas as músicas.
- Genre: Gênero da música.
- Genres: Gêneros possíveis.

#### Verificando o gênero musical dos dados. Todos são `sertanejo`

In [15]:
df_sertanejo['Genre'].value_counts()[0:10]

Sertanejo    15381
Name: Genre, dtype: int64

#### Verificando os artistas do dataset, é possível observar que o artista com mais músicas é a dupla `Teodoro e Sampaio`

In [16]:
df_sertanejo['Artist'].value_counts()[0:10]

Teodoro e Sampaio            432
Paula Fernandes              394
Tião Carreiro e Pardinho     378
Milionário e José Rico       370
Zezé Di Camargo e Luciano    368
Luan Santana                 332
Lourenço & Lourival          306
Chitãozinho e Xororó         299
Rick & Renner                297
Bruno e Marrone              275
Name: Artist, dtype: int64

#### Pré-processamento do texto

Procurar estilos de letra por conjuntos de músicas sertanejas que são semelhantes. Serão utilizadas as letras das músicas pra fazer o agrupamento

Removendo ruídos - `stopwords`

In [17]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

207


In [18]:
#construindo uma nova lista para armazenar o texto limpo
clean_lyrics = []
for w in range(len(df_sertanejo.Lyric)):
    lyric = df_sertanejo['Lyric'][w]

    # removendo caracteres especiais e dígitos
    # removendo palavras com dois ou menos caracteres e alguns acordes contidos no dataset
    lyric = re.sub("(\\d|\\W)+|\w*\d\w*", " ", lyric)
    lyric = ' '. join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_lyrics.append(lyric)

clean_lyrics[0:5]

['Não vou mais pensar você minha mente não vai entrar vim aqui avisar Teu psicológico preparar Que não vou mais esperar vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que você não deu valor sua única chance vai ser alguma balada vida beijar sem perceber Sem ver que você avisei Tenta não vacilar Menina avisei Amor pra cuidar escapar mão cai chão Não tem conserto não Antes sumir sua vida mais uma coisa aqui portão última vez que vou falar Seu psicológico preparar Que não vou mais esperar Acabou minha paciência vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que você não deu valor sua única chance Você não deu valor Você não deu valor Não deu valor não deu valor Amei demais agora não quero mais nem saber Deixa outras aproveitarem por você vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que 

#### Vetores TF-IDF

Transformar o texto num vetor de características que serão utilizadas na comparação

In [20]:
#TF_IDF vetorização
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features=None,
    stop_words=stop_words,
    ngram_range=(1,3)
)

#transformação
vec_text = tfv.fit_transform(clean_lyrics)

#retorna a lista de palavras
words = tfv.get_feature_names()

len(words)

46139

In [21]:
words[1:10]

['aah',
 'aah aah',
 'aahh',
 'aai',
 'aba',
 'aba chapéu',
 'aba larga',
 'aba larga bruaca',
 'abafado']

# Agrupamento

#### Decidindo o K

In [24]:
#escolhendo k
#sdd - soma das distâncias quadráticas ao centro do grupo
qualidade = pd.DataFrame(columns=['k', 'ssd'])
for k in range(1,17, 1):
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

C:\Users\rayan\AppData\Local\Temp\ipykernel_10236\540217079.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_10236\540217079.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_10236\540217079.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_10236\540217079.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

alt.Chart(...)

A partir do gráfico plotado anteriormente, é possível observar que entre os pontos 13 e 15 existe uma constância, por tanto, vou utilizar o valor 13 como quantidade dos centróides

In [30]:
kmeans = MiniBatchKMeans(n_clusters=13, init_size=1024, batch_size=2048, random_state=20)
#fit nos dados
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

df_sertanejo_ag = df_sertanejo.assign(grupo = labels)

df_sertanejo_ag.sample(3)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,grupo
1266,Só Dava Eu,Mais uma foto com o filtro bacana. Frase de ef...,César Menotti e Fabiano,239,3.0,Sertanejo,Sertanejo,9
14273,Boa Sorte Pra Você,Pra você não foi sério. E se ainda te quero. S...,Victor e Leo,181,8.0,Sertanejo,Sertanejo,1
3532,Eu Amo Você,"Os olhos de um cara apaixonado,. É um circo il...",Eduardo Costa,267,7.9,Sertanejo,Sertanejo; Romântico; Forró; Trilha Sonora; Co...,8


#### Interpretando os grupos

In [31]:
df_sertanejo_ag['grupo'].value_counts()

1     4376
9     2576
11    2289
7     1065
0      968
12     937
4      760
8      709
3      502
10     441
6      420
5      209
2      129
Name: grupo, dtype: int64

1. Utilize o k-means para procurar grupos de músicas, identificando o melhor número de grupos através das técnicas explicadas nas aulas. Em seguida, descreva os grupos encontrados, listando as palavras e os artistas mais frequentes em cada grupo. Tente nomear os grupos e explique em detalhes o racional para sua nomeação para cada grupo. Repita este processo utilizando como entrada para o k-means:
- O título da música;
- A letra da música;
- A concatenação do título e letra;

In [29]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '. join(words[word] for word in centroid))

0 : pra, mim, pra mim, quero, noite, amor, assim, tudo, coração, vai, faz, vou, dia, sei
1 : amor, pra, sei, coração, quero, tudo, mim, vida, tão, amar, assim, tempo, saudade, vou
2 : you, the, love, and, amor, love you, your, baby, esquecer, love love, inhambu, fiquei, xororó, pra
3 : maria, pedro, santa maria, santa, paulo, pedro paulo mariano, paulo mariano, pedro paulo, maria serra, mariano, paulo mariano santa, mariano santa, mariano santa maria, santa maria serra
4 : paixão, amor, coração, homem, vida, dor, saudade, peito, triste, pra, solidão, outro, mulher, sempre
5 : chora, dói, chora chora, viola, dói dói, coração, saudade, amor, ver, pra, peito, vai, homem chora, homem
6 : vem, vem vem, amor, pra, quero, vai, vem vem vem, coração, comigo, vou, amar, boca, mim, aqui
7 : vou, pra, amor, quero, vai, hoje, fazer, sei, chorar, coração, ficar, vida, tudo, agora
8 : amor, amor amor, pode, pra, coração, quero, faz, assim, paixão, vida, amar, mim, amor amor amor, tudo
9 : gente, pra,

Como dito anteriormente, foram construídos 13 clusters que estão dividos em grupos de 0 a 12.

- o grupo 0 fala da vontade da pessoa. do querer algo ou fazer algo com alguém.
- os grupos 1, 4, 5 é um sertanejo sofrência, pois fala muito em saudade, amor, coração, dor, tristeza, solidão
- o grupo 2 fala de amor usando termos em inglês.
- o grupo 3 aparentemente é um grupo de sertanejo católico, pois fala bastante em santos e santas e também utiliza bastante o termo **mariano**. termo muito utilizado no catolicismo em referência à Santa Maria, mãe de Deus.
- os grupos 6, 7 é um sertanejo romântico que expressa o sentimento e a vontade pelo outro.
os grupos 8 e 9 falam de amor, coração, paixão.
- o grupo 10 fala das características da mulher, do amor do peão pela mulher bonita.
- o grupo 11 também parece ser um grupo de sertanejo católico, que fala de Deus e Jesus, mas no sentido de conexão Pai e filho, de cuidado, proteção com a vida, teraa, mundo, sertão.
o grupo 12 fala do querer imediato do coração. como se expressasse uma vontade imediata de ficar junto de quem se gosta.

In [32]:
pd.options.display.max_colwidth=100
df_sertanejo_ag.query('grupo == 5')[['SName', 'Lyric', 'Artist']].sample(5)

,SName,Lyric,Artist
13283,A Gente Chora,"E a gente chora. Mas quando chora de saudade, a gente foge. Pra se encontrar em outra cidade. E ...",Thaeme e Thiago
11886,Coração Chora de Saudade,"Quando a noite chega ao fim. E aproxima a madrugada. Tentando fugir de mim,. Lá vou eu pelas cal...",Rionegro & Solimões
4650,Dói Demais,"Ai, ai, vamos chorar nossa mágoa qual meu coração. Ai, ai, não pra aguentar sem choro tamanha pa...",Gilberto e Gilmar
1654,Feitiço Espanhol,"Ela chegou com o circo.. Dedilhava castanholas.. Era morena e bonita.. Era meiga, era espanhola....",Chico Rey e Paraná
3297,Último Freguês,Quantas vezes na mesa do bar. Eu adormeci esperando meu bem. Sabendo que ela não voltava. Porque...,Di Paullo e Paulino


In [39]:
for g in range(0, 13):
    print('\n-----\nGrupo {}:'.format(g))
    print(df_sertanejo_ag.query('grupo == {}'. format(g))['Artist'].value_counts()[0:10])
    print('-------')


-----
Grupo 0:
Zezé Di Camargo e Luciano    34
Paula Fernandes              30
Teodoro e Sampaio            25
Luan Santana                 24
Guilherme e Santiago         24
Gusttavo Lima                23
Jorge e Mateus               22
Rick & Renner                22
Gilberto e Gilmar            22
Roberta Miranda              22
Name: Artist, dtype: int64
-------

-----
Grupo 1:
Paula Fernandes              158
Zezé Di Camargo e Luciano    152
Milionário e José Rico       144
João Mineiro e Marciano      129
Luan Santana                 128
Roberta Miranda              115
Bruno e Marrone              112
Matogrosso e Mathias         110
Chitãozinho e Xororó         109
Leonardo                     109
Name: Artist, dtype: int64
-------

-----
Grupo 2:
Paula Fernandes              16
Victor e Leo                 10
Leonardo                      9
Milionário e José Rico        8
Chitãozinho e Xororó          7
Luan Santana                  6
Zezé Di Camargo e Luciano     6
Tonico e

Paula Fernandes parece ser a artisya que mais parece na maioria dos grupos, seguida por Zezé di Camargo e Luciano

Aqui podemos ver a quantidade de músicas que o `Artist == "Bruno e Marrone` tem em determinado cluster

In [34]:
df_sertanejo_ag.query('Artist == "Bruno e Marrone"')[['grupo', 'SName']].groupby('grupo').count()

,SName
grupo,
0,18
1,112
2,4
4,9
5,9
6,9
7,27
8,14
9,34


#### Visualizando os grupos em muitas dimensões

In [36]:
df_sertanejo_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 15381 samples in 0.013s...
[t-SNE] Computed neighbors for 15381 samples in 13.048s...
[t-SNE] Computed conditional probabilities for sample 1000 / 15381
[t-SNE] Computed conditional probabilities for sample 2000 / 15381
[t-SNE] Computed conditional probabilities for sample 3000 / 15381
[t-SNE] Computed conditional probabilities for sample 4000 / 15381
[t-SNE] Computed conditional probabilities for sample 5000 / 15381
[t-SNE] Computed conditional probabilities for sample 6000 / 15381
[t-SNE] Computed conditional probabilities for sample 7000 / 15381
[t-SNE] Computed conditional probabilities for sample 8000 / 15381
[t-SNE] Computed conditional probabilities for sample 9000 / 15381
[t-SNE] Computed conditional probabilities for sample 10000 / 15381
[t-SNE] Computed conditional probabilities for sample 11000 / 15381
[t-SNE] Computed conditional probabilities for sample 12000 / 15381
[t-SNE] Computed conditional probabilities for s

In [50]:
df_sertanejo_ag = df_sertanejo_ag.assign(pca1 = df_sertanejo_embedded[:,0], pca2 = df_sertanejo_embedded[:,1])

alt.Chart(df_sertanejo_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'pca1',
    y = 'pca2',
    color = 'grupo:N',
    tooltip=['Artist', "SName"]
).interactive()

c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

2. Mesmo que 1 acima, mas aplique uma redução de dimensionalidade utilizando o PCA, e utilize os dados transformados como entrada para o k-means.

#### PCA - Principal Component Analysis

![k-means.png](pca.png)

Redução de Dimensionalidade

A Análise de Componentes Principais ou PCA (Principal Component Analysis) é uma técnica de análise multivariada que pode ser usada para analisar inter-relações entre um grande número de variáveis e explicar essas variáveis em termos de suas dimensões inerentes (Componentes).

O objetivo é encontrar um meio de condensar a informação contida em várias variáveis originais em um conjunto menor de variáveis estatísticas (componentes) com uma perda mínima de informação.

O número de componentes principais se torna o número de variáveis consideradas na análise, mas geralmente as primeiras componentes são as mais importantes já que explicam a maior parte da variação total.

As componentes principais em geral são extraídas via matriz de covariância, mas também podem ser extraídas via matriz de correlação.

Fonte: https://site.statplace.com.br/blog/analise-de-componentes-principais/

In [44]:
pca = PCA(n_components=2)
pca.fit(df_sertanejo_embedded)

PCA(n_components=2)

In [45]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.547175   0.45282495]
[5262.795 4787.607]


In [49]:
alt.Chart(df_sertanejo_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2',
    color = 'grupo:N',
    tooltip=['Artist', "SName"]
).interactive()

alt.Chart(...)